In [1]:
import os
HOME = os.getcwd()
print(HOME)

import sys
#from google.colab.patches import cv2_imshow
import cv2
import PIL
from PIL import Image
import numpy as np
import uuid
import tempfile

from IPython import display
display.clear_output()
from IPython.display import display, Image
from io import BytesIO

In [2]:
print(HOME)

/home/michal/Documents/FIT/DP/dp


In [3]:
VIDEO_PATH = "skateboarding.mp4"

In [4]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.1.6 🚀 Python-3.11.5 torch-2.1.2+cu121 CPU (AMD Ryzen 5 5625U with Radeon Graphics)
Setup complete ✅ (12 CPUs, 30.7 GB RAM, 198.2/915.3 GB disk)


In [5]:
# Instantiate YOLOv8 model

model = YOLO(f'{HOME}/yolov8n.pt')
colors = np.random.randint(0, 256, size=(len(model.names), 3))

print(model.names)

# Specify which classes you care about. The rest of classes will be filtered out.
chosen_class_ids = [0] # person

{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

In [19]:
# Import packages

import torch
import matplotlib.pyplot as plt
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

In [22]:
sam_model = "weights/sam_vit_h_4b8939.pth"
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
sam = sam_model_registry["vit_h"](checkpoint=sam_model).to(device=DEVICE)
mask_predictor = SamPredictor(sam)

In [8]:
# Cast color to ints
def get_color(color):
    return (int(color[0]), int(color[1]), int(color[2]))

# Get video dimensions
def get_video_dimensions(input_cap):
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    return height, width

# Get output video writer with same dimensions and fps as input video
def get_output_video_writer(input_cap, output_path):
    # Get the video's properties (width, height, FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

  # Define the output video file
    output_codec = cv2.VideoWriter_fourcc(*"mp4v")  # MP4 codec
    output_video = cv2.VideoWriter(output_path, output_codec, fps, (width, height))

    return output_video

# Visualize a video frame with bounding boxes, classes and confidence scores
def visualize_detections(frame, boxes, conf_thresholds, class_ids):
    frame_copy = np.copy(frame)
    for idx in range(len(boxes)):
        class_id = int(class_ids[idx])
        conf = float(conf_thresholds[idx])
        x1, y1, x2, y2 = int(boxes[idx][0]), int(boxes[idx][1]), int(boxes[idx][2]), int(boxes[idx][3])
        color = colors[class_id]
        label = f"{model.names[class_id]}: {conf:.2f}"
        cv2.rectangle(frame_copy, (x1, y1), (x2, y2), get_color(color), 2)
        cv2.putText(frame_copy, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, get_color(color), 2)
    return frame_copy

def add_color_to_mask(mask, color):
    next_mask = mask.astype(np.uint8)
    next_mask = np.expand_dims(next_mask, 0).repeat(3, axis=0)
    next_mask = np.moveaxis(next_mask, 0, -1)
    return next_mask * color

# Merge masks into a single, multi-colored mask
def merge_masks_colored(masks, class_ids):
    filtered_class_ids = []
    filtered_masks = []
    for idx, cid in enumerate(class_ids):
        if int(cid) in chosen_class_ids:
            filtered_class_ids.append(cid)
            filtered_masks.append(masks[idx])

    merged_with_colors = add_color_to_mask(filtered_masks[0][0], get_color(colors[int(filtered_class_ids[0])])).astype(np.uint8)

    if len(filtered_masks) == 1:
        return merged_with_colors

    for i in range(1, len(filtered_masks)):
        curr_mask_with_colors = add_color_to_mask(filtered_masks[i][0], get_color(colors[int(filtered_class_ids[i])]))
        merged_with_colors = np.bitwise_or(merged_with_colors, curr_mask_with_colors)

    return merged_with_colors.astype(np.uint8)

def get_instance_uri(client, global_key, array):
    """ Reads a numpy array into a temp Labelbox data row to-be-uploaded to Labelbox
    Args:
        client        :   Required (lb.Client) - Labelbox Client object
        global_key    :   Required (str) - Data row global key
        array         :   Required (np.ndarray) - NumPy ndarray representation of an image
    Returns:
        Temp Labelbox data row to-be-uploaded to Labelbox as row data
    """
    # Convert array to PIL image
    image_as_pil = PIL.Image.fromarray(array)
    # Convert PIL image to PNG file bytes
    image_as_bytes = BytesIO()
    image_as_pil.save(image_as_bytes, format='PNG')
    image_as_bytes = image_as_bytes.getvalue()
    # Convert PNG file bytes to a temporary Labelbox URL
    url = client.upload_data(
        content=image_as_bytes, 
        filename=global_key,
        content_type="image/jpeg",
        sign=True
    )
    # Return the URL
    return url

def get_local_instance_uri(array):
    # Convert array to PIL image
    image_as_pil = PIL.Image.fromarray(array)

    with tempfile.NamedTemporaryFile(suffix='.png', dir="/content", delete=False) as temp_file:
        image_as_pil.save(temp_file)
        file_name = temp_file.name

    # Return the URL
    return file_name

def create_mask_frame(frame_num, instance_uri):
    return lb_types.MaskFrame(index=frame_num, instance_uri=instance_uri)

def create_mask_instances(class_ids):
    instances = []
    for cid in list(set(class_ids)): # get unique class ids
        if int(cid) in chosen_class_ids:
            color = get_color(colors[int(cid)])
            name = model.names[int(cid)]
            instances.append(lb_types.MaskInstance(color_rgb=color, name=name))
    return instances

def create_video_mask_annotation(frames, instance):
    return lb_types.VideoMaskAnnotation(
        frames=frames,
        instances=[instance]
    )

In [14]:
# Run through YOLOv8 on the video once quickly to get unique class ids present
# This will inform which classes we add to the ontology

cap = cv2.VideoCapture(VIDEO_PATH)

unique_class_ids = set()

# Loop through the frames of the video
frame_num = 1
while cap.isOpened():
    if frame_num % 30 == 0 or frame_num == 1:
        print("Processing frame number", frame_num)
    ret, frame = cap.read()
    if not ret:
        break

  # Run frame through YOLOv8 and get class ids predicted
    detections = model.predict(frame, conf=0.7) # frame is a numpy array
    for cid in detections[0].boxes.cls:
        unique_class_ids.add(int(cid))
        frame_num += 1

cap.release()

Processing frame number 1

0: 384x640 (no detections), 42.0ms
Speed: 2.4ms preprocess, 42.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 1

0: 384x640 (no detections), 42.1ms
Speed: 1.3ms preprocess, 42.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 1

0: 384x640 (no detections), 38.8ms
Speed: 1.6ms preprocess, 38.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 1

0: 384x640 (no detections), 37.8ms
Speed: 1.2ms preprocess, 37.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 1

0: 384x640 (no detections), 40.2ms
Speed: 1.5ms preprocess, 40.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 1

0: 384x640 (no detections), 40.4ms
Speed: 1.5ms preprocess, 40.4ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 1

0: 384x640 (no dete

Speed: 1.5ms preprocess, 40.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.1ms
Speed: 1.7ms preprocess, 40.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 skateboard, 38.7ms
Speed: 1.4ms preprocess, 38.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.4ms
Speed: 1.5ms preprocess, 40.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.9ms
Speed: 1.7ms preprocess, 31.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 33.8ms
Speed: 1.6ms preprocess, 33.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 43.3ms
Speed: 1.7ms preprocess, 43.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 41.9ms
Speed: 1.7ms preprocess, 41.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384


0: 384x640 1 person, 39.4ms
Speed: 1.6ms preprocess, 39.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 30.8ms
Speed: 1.8ms preprocess, 30.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.5ms
Speed: 1.5ms preprocess, 38.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.6ms
Speed: 1.5ms preprocess, 39.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.6ms
Speed: 1.5ms preprocess, 38.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.9ms
Speed: 1.5ms preprocess, 39.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.5ms
Speed: 1.5ms preprocess, 38.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 skateboard, 39.2ms
Speed: 1.5ms preprocess, 39.2ms inference, 0.7ms postprocess per image at s


0: 384x640 1 person, 38.6ms
Speed: 1.4ms preprocess, 38.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 32.3ms
Speed: 1.3ms preprocess, 32.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 32.3ms
Speed: 1.7ms preprocess, 32.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.6ms
Speed: 1.8ms preprocess, 31.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.5ms
Speed: 1.2ms preprocess, 31.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 32.8ms
Speed: 1.7ms preprocess, 32.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 32.3ms
Speed: 1.7ms preprocess, 32.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)
Processing frame number 150

0: 384x640 1 person, 30.7ms
Speed: 1.9ms preprocess, 30.7ms inference, 0.5ms postprocess 


0: 384x640 1 person, 38.6ms
Speed: 1.4ms preprocess, 38.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.6ms
Speed: 1.4ms preprocess, 38.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 37.9ms
Speed: 1.4ms preprocess, 37.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.8ms
Speed: 1.4ms preprocess, 38.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.2ms
Speed: 1.4ms preprocess, 39.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.4ms
Speed: 1.4ms preprocess, 39.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.2ms
Speed: 1.4ms preprocess, 38.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.5ms
Speed: 1.6ms preprocess, 40.5ms inference, 0.6ms postprocess per image at shape (1, 3, 38

...


0: 384x640 1 person, 31.8ms
Speed: 1.3ms preprocess, 31.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 32.8ms
Speed: 1.6ms preprocess, 32.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 34.6ms
Speed: 1.5ms preprocess, 34.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 30.7ms
Speed: 1.6ms preprocess, 30.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 30.4ms
Speed: 1.6ms preprocess, 30.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.1ms
Speed: 1.8ms preprocess, 40.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.2ms
Speed: 1.5ms preprocess, 39.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.6ms
Speed: 1.5ms preprocess, 38.6ms inference, 0.6ms postprocess per image at shape (1, 3, 38


0: 384x640 1 person, 39.6ms
Speed: 1.5ms preprocess, 39.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.6ms
Speed: 1.5ms preprocess, 39.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.4ms
Speed: 1.6ms preprocess, 39.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.6ms
Speed: 1.4ms preprocess, 39.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 37.9ms
Speed: 1.5ms preprocess, 37.9ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.1ms
Speed: 1.5ms preprocess, 38.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.8ms
Speed: 1.5ms preprocess, 38.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.1ms
Speed: 1.4ms preprocess, 39.1ms inference, 0.6ms postprocess per image at shape (1, 3, 38


0: 384x640 (no detections), 36.5ms
Speed: 1.4ms preprocess, 36.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.3ms
Speed: 1.5ms preprocess, 40.3ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.8ms
Speed: 1.5ms preprocess, 38.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.7ms
Speed: 1.4ms preprocess, 38.7ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.6ms
Speed: 1.4ms preprocess, 38.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.4ms
Speed: 1.7ms preprocess, 38.4ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.0ms
Speed: 1.6ms preprocess, 39.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 30.8ms
Speed: 1.4ms preprocess, 30.8ms inference, 0.6ms postprocess per image at shape (1


0: 384x640 1 person, 41.2ms
Speed: 1.5ms preprocess, 41.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.1ms
Speed: 1.5ms preprocess, 40.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.4ms
Speed: 1.6ms preprocess, 39.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 37.8ms
Speed: 1.5ms preprocess, 37.8ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.9ms
Speed: 1.5ms preprocess, 40.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.4ms
Speed: 1.5ms preprocess, 40.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 39.9ms
Speed: 1.5ms preprocess, 39.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.0ms
Speed: 1.5ms preprocess, 40.0ms inference, 0.6ms postprocess per image at shape (1, 3, 38


0: 384x640 1 person, 31.5ms
Speed: 1.5ms preprocess, 31.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.3ms
Speed: 1.6ms preprocess, 40.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.1ms
Speed: 1.3ms preprocess, 38.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.2ms
Speed: 1.4ms preprocess, 38.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.2ms
Speed: 1.4ms preprocess, 40.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.6ms
Speed: 1.4ms preprocess, 38.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 40.1ms
Speed: 2.0ms preprocess, 40.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 30.8ms
Speed: 1.6ms preprocess, 30.8ms inference, 0.5ms postprocess per image at shape (1, 3, 38

In [12]:
# Run through YOLOv8 on the video once quickly to get unique class ids present
# This will inform which classes we add to the ontology

cap = cv2.VideoCapture(VIDEO_PATH)

unique_class_ids = set()

# Loop through the frames of the video
frame_num = 1
while cap.isOpened():
    if frame_num % 30 == 0 or frame_num == 1:
        print("Processing frame number", frame_num)
    ret, frame = cap.read()
    print(type(frame), frame)
    if frame_num == 20:
        break
    if not ret:
        break

  # Run frame through YOLOv8 and get class ids predicted
    detections = model.predict(frame, conf=0.7) # frame is a numpy array
    for cid in detections[0].boxes.cls:
        unique_class_ids.add(int(cid))
        frame_num += 1

cap.release()

Processing frame number 1


error: OpenCV(4.9.0) :-1: error: (-5:Bad argument) in function 'imshow'
> Overload resolution failed:
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)
>  - imshow() missing required argument 'mat' (pos 2)


In [15]:
unique_class_ids

{0, 2, 36}

In [24]:
# Run YOLOv8 and then SAM on each frame, and write visualization videos to disk
# You can download /content/skateboarding_boxes.mp4 and /content/skateboarding_masks.mp4
# to visualize the results

cap = cv2.VideoCapture(VIDEO_PATH)

output_video_boxes = get_output_video_writer(cap, "skateboarding_boxes.mp4")
output_video_masks = get_output_video_writer(cap, "skateboarding_masks.mp4")
mask_frames = []

# Loop through the frames of the video
frame_num = 1
while cap.isOpened():
    if frame_num % 30 == 0 or frame_num == 1:
        print("Processing frames", frame_num, "-", frame_num+29)
    ret, frame = cap.read()
    if not ret:
        break

  # Run frame through YOLOv8 to get detections
    detections = model.predict(frame, conf=0.7) # frame is a numpy array
  
  # Write detections to output video
    frame_with_detections = visualize_detections(frame, 
                                                 detections[0].boxes.cpu().xyxy, 
                                                 detections[0].boxes.cpu().conf, 
                                                 detections[0].boxes.cpu().cls)
    output_video_boxes.write(frame_with_detections)
    
    
    # Run frame and detections through SAM to get masks
    transformed_boxes = mask_predictor.transform.apply_boxes_torch(detections[0].boxes.xyxy, list(get_video_dimensions(cap)))
    if len(transformed_boxes) == 0:
        print("No boxes found on frame", frame_num)
        output_video_masks.write(frame)
        frame_num += 1
        continue
    mask_predictor.set_image(frame)
    masks, scores, logits = mask_predictor.predict_torch(
        boxes = transformed_boxes,
        multimask_output=False,
        point_coords=None,
        point_labels=None
    )
    masks = np.array(masks.cpu())
    if masks is None or len(masks) == 0:
        print("No masks found on frame", frame_num)
        output_video_masks.write(frame)
        frame_num += 1
        continue
    merged_colored_mask = merge_masks_colored(masks, detections[0].boxes.cls)
  
  # Write masks to output video
    image_combined = cv2.addWeighted(frame, 0.7, merged_colored_mask, 0.7, 0)
    output_video_masks.write(image_combined)
    
    frame_num += 1

  # For the purposes of this demo, only look at the first 90 frames
    if frame_num > 90:
        break

cap.release()
output_video_boxes.release()
output_video_masks.release()
cv2.destroyAllWindows()

Processing frames 1 - 30

0: 384x640 (no detections), 28.3ms
Speed: 1.3ms preprocess, 28.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
No boxes found on frame 1

0: 384x640 (no detections), 37.9ms
Speed: 1.4ms preprocess, 37.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
No boxes found on frame 2

0: 384x640 (no detections), 29.0ms
Speed: 2.1ms preprocess, 29.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
No boxes found on frame 3

0: 384x640 (no detections), 30.0ms
Speed: 1.6ms preprocess, 30.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
No boxes found on frame 4

0: 384x640 (no detections), 39.5ms
Speed: 1.7ms preprocess, 39.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
No boxes found on frame 5

0: 384x640 (no detections), 38.2ms
Speed: 1.4ms preprocess, 38.2ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
No boxes found on frame 6

0: 384x640 (no detec


0: 384x640 1 person, 30.0ms
Speed: 1.2ms preprocess, 30.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.2ms
Speed: 2.2ms preprocess, 31.2ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 skateboard, 28.0ms
Speed: 1.4ms preprocess, 28.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.1ms
Speed: 1.6ms preprocess, 38.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 31.1ms
Speed: 2.0ms preprocess, 31.1ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 28.2ms
Speed: 1.6ms preprocess, 28.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Processing frames 60 - 89

0: 384x640 1 person, 29.2ms
Speed: 1.2ms preprocess, 29.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 38.5ms
Speed: 2.2ms preprocess, 38.5ms inference, 0.6ms 